In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import csv
from datetime import datetime
import feather

In [2]:
!ls

'Topic Modeling.ipynb'		  model5.gensim
 corpus.pkl			  model5.gensim.expElogbeta.npy
 df_topic_model.csv		  model5.gensim.id2word
 dictionary.gensim		  model5.gensim.state
 download_clean.py		  mongo_export.sh
 generate_general_fan.ipynb	 'pymongo explore mongodb.ipynb'
 gpt_2				  sample.csv
 m				  series_topic_model.csv
 model10.gensim			  tester
 model10.gensim.expElogbeta.npy   vader_sentiment_analysis.ipynb
 model10.gensim.id2word		  vader_sentiment_by_nfl_team.ipynb
 model10.gensim.state


In [3]:
train = pd.read_csv("sample.csv")

/home/ubuntu/anaconda3/envs/ds_env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
train.shape 

(8742798, 21)

In [5]:
train.head()

,_id,author,author_flair_text,body,can_gild,collapsed,collapsed_reason,controversiality,created_utc,distinguished,...,gilded,id,is_submitter,link_id,parent_id,retrieved_on,score,stickied,subreddit,subreddit_id
0,ObjectId(5d585f95a772b7432fc82a65),NaN,NaN,A lot of anecdotes about a fired up defense ch...,True,False,NaN,0,1501545604,NaN,...,0,dkzncdh,False,t3_6qs8v7,t3_6qs8v7,1503654248,26,False,nyjets,t5_2s2av
1,ObjectId(5d585f95a772b7432fc82b20),NaN,NaN,Nice. So based on this positive report he shou...,True,False,NaN,0,1501545609,NaN,...,0,dkzncis,False,t3_6qqjcb,t3_6qqjcb,1503654250,-3,False,CHIBears,t5_2s0u3
2,ObjectId(5d585f95a772b7432fc82c9b),NaN,NaN,"Future salaries would be a concern, but that's...",True,False,NaN,0,1501545620,NaN,...,0,dkznctk,False,t3_6qrcmr,t1_dkzjtoa,1503654254,3,False,cowboys,t5_2re7g
3,ObjectId(5d585f95a772b7432fc82de8),NaN,NaN,PFF generally graded him as a below-average st...,True,False,NaN,0,1501545629,NaN,...,0,dkznd2x,False,t3_6qs0d8,t3_6qs0d8,1503654257,8,False,Texans,t5_2s450
4,ObjectId(5d585f95a772b7432fc8304f),NaN,NaN,"dude, you gotta risk it for the bojangles bisc...",True,False,NaN,0,1501545646,NaN,...,0,dkzndk9,False,t3_6qrzaa,t3_6qrzaa,1503654265,18,False,panthers,t5_2s1w0


In [6]:
# train.drop('0',axis=1,inplace=True)

In [7]:
train.shape

(8742798, 21)

In [8]:
train.columns

Index(['_id', ' author', ' author_flair_text', 'body', 'can_gild', 'collapsed',
       'collapsed_reason', 'controversiality', 'created_utc', 'distinguished',
       'edited', 'gilded', 'id', 'is_submitter', 'link_id', 'parent_id',
       'retrieved_on', 'score', 'stickied', 'subreddit', 'subreddit_id'],
      dtype='object')

In [9]:
analyser = SentimentIntensityAnalyzer()

In [10]:
def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    print("{:-<40} {}".format(sentence, str(score)))
    return score

In [11]:
scores = sentiment_analyzer_scores("The service here is good")

The service here is good---------------- {'neg': 0.0, 'neu': 0.58, 'pos': 0.42, 'compound': 0.4404}


In [12]:
scores

{'neg': 0.0, 'neu': 0.58, 'pos': 0.42, 'compound': 0.4404}

In [13]:
def create_sentiment(row,col):
    sentence = row
    score = analyser.polarity_scores(sentence)
    scores = score[col]
    return scores
#     row['neu'] = score['neu']
#     row['pos'] = score['pos']
#     row['compound'] = score['compound']
    

In [14]:
# for sent in ['neg','neu','pos','compound']:
#     train[sent] = train['body'].apply(lambda x: create_sentiment(x,sent))
    

In [15]:
train.head()

,_id,author,author_flair_text,body,can_gild,collapsed,collapsed_reason,controversiality,created_utc,distinguished,...,parent_id,retrieved_on,score,stickied,subreddit,subreddit_id,neg,neu,pos,compound
0,ObjectId(5d585f95a772b7432fc82a65),NaN,NaN,A lot of anecdotes about a fired up defense ch...,True,False,NaN,0,1501545604,NaN,...,t3_6qs8v7,1503654248,26,False,nyjets,t5_2s2av,0.108,0.561,0.331,0.8519
1,ObjectId(5d585f95a772b7432fc82b20),NaN,NaN,Nice. So based on this positive report he shou...,True,False,NaN,0,1501545609,NaN,...,t3_6qqjcb,1503654250,-3,False,CHIBears,t5_2s0u3,0.130,0.530,0.340,0.6542
2,ObjectId(5d585f95a772b7432fc82c9b),NaN,NaN,"Future salaries would be a concern, but that's...",True,False,NaN,0,1501545620,NaN,...,t1_dkzjtoa,1503654254,3,False,cowboys,t5_2re7g,0.183,0.619,0.198,0.0772
3,ObjectId(5d585f95a772b7432fc82de8),NaN,NaN,PFF generally graded him as a below-average st...,True,False,NaN,0,1501545629,NaN,...,t3_6qs0d8,1503654257,8,False,Texans,t5_2s450,0.148,0.702,0.150,0.3071
4,ObjectId(5d585f95a772b7432fc8304f),NaN,NaN,"dude, you gotta risk it for the bojangles bisc...",True,False,NaN,0,1501545646,NaN,...,t3_6qrzaa,1503654265,18,False,panthers,t5_2s1w0,0.149,0.851,0.000,-0.2732


In [16]:
def convert_utc_to_normal_time(row):
    time = datetime.fromtimestamp(row).strftime('%Y-%m-%d %H:%M:%S')
    return time

In [17]:
train['datetime'] = train['created_utc'].apply(convert_utc_to_normal_time)


In [18]:
train.head()

,_id,author,author_flair_text,body,can_gild,collapsed,collapsed_reason,controversiality,created_utc,distinguished,...,retrieved_on,score,stickied,subreddit,subreddit_id,neg,neu,pos,compound,datetime
0,ObjectId(5d585f95a772b7432fc82a65),NaN,NaN,A lot of anecdotes about a fired up defense ch...,True,False,NaN,0,1501545604,NaN,...,1503654248,26,False,nyjets,t5_2s2av,0.108,0.561,0.331,0.8519,2017-08-01 00:00:04
1,ObjectId(5d585f95a772b7432fc82b20),NaN,NaN,Nice. So based on this positive report he shou...,True,False,NaN,0,1501545609,NaN,...,1503654250,-3,False,CHIBears,t5_2s0u3,0.130,0.530,0.340,0.6542,2017-08-01 00:00:09
2,ObjectId(5d585f95a772b7432fc82c9b),NaN,NaN,"Future salaries would be a concern, but that's...",True,False,NaN,0,1501545620,NaN,...,1503654254,3,False,cowboys,t5_2re7g,0.183,0.619,0.198,0.0772,2017-08-01 00:00:20
3,ObjectId(5d585f95a772b7432fc82de8),NaN,NaN,PFF generally graded him as a below-average st...,True,False,NaN,0,1501545629,NaN,...,1503654257,8,False,Texans,t5_2s450,0.148,0.702,0.150,0.3071,2017-08-01 00:00:29
4,ObjectId(5d585f95a772b7432fc8304f),NaN,NaN,"dude, you gotta risk it for the bojangles bisc...",True,False,NaN,0,1501545646,NaN,...,1503654265,18,False,panthers,t5_2s1w0,0.149,0.851,0.000,-0.2732,2017-08-01 00:00:46


In [19]:
def get_state(row):
    if row in ['Colts']:
        return "IN"
    if row in ['DenverBroncos']:
        return "CO"
    if row in ['buffalobills',"nyjets","NYGiants"]:
        return "NY"
    if row in ['ravens',"Redskins"]:
        return "MD"
    if row in ['Patriots']:
        return "MA"
    if row in ['bengals',"Browns"]:
        return "OH"
    if row in ['miamidolphins',"buccaneers","Jaguars"]:
        return "FL"
    if row in ['KansasCityChiefs']:
        return "KS"
    if row in ['Texans',"cowboys"]:
        return "TX"
    if row in ['oaklandraiders',"49ers","LosAngelesRams","Chargers"]:
        return "CA"
    if row in ['steelers',"eagles"]:
        return "PA"
    if row in ['Tennesseetitans']:
        return "TN"
    if row in ['CHIBears']:
        return "IL"
    if row in ['falcons']:
        return "GA"
    if row in ['AZCardinals']:
        return "AZ"
    if row in ['detroitlions']:
        return "MI"
    if row in ['panthers']:
        return "SC"
    if row in ['GreenBayPackers']:
        return "WI"
    if row in ["Saints"]:
        return "LA"
    if row in ["Seahawks"]:
        return "WA"
    if row in ['minnesotavikings']:
        return "MN"


In [20]:
train['state'] = train['subreddit'].apply(get_state)

In [22]:
import feather

In [23]:
feather.write_dataframe(train, "nfl_processed.feather")

In [24]:
def get_lat_long(row):
    if row=='Colts':
        return (39.760056,-86.163806)
    if row=='DenverBroncos':
        return (39.7439,-105.020097)
    if row=='buffalobills':
        return (42.773739,-78.786978)
    if row=="nyjets":
        return (40.812194,-74.076983)
    if row=="NYGiants":
        return (40.812194,-74.076983)
    if row=='ravens':
        return (39.277969,-76.622767)
    if row=="Redskins":
        return (38.907697,-76.864517)
    if row in ['Patriots']:
        return (42.090925,-71.26435)
    if row=='bengals':
        return (39.095442,-84.516039)
    if row=="Browns":
        return (41.506022,-81.699564)
    if row=='miamidolphins':
        return (25.957919,-80.238842)
    if row=="buccaneers":
        return (27.975967,-82.50335)
    if row=="Jaguars":
        return (30.323925,-81.637356)
    if row=='KansasCityChiefs':
        return (39.048914,-94.484039)
    if row=='Texans':
        return (29.684781,-95.410956)
    if row=="cowboys":
        return (32.747778,-97.092778)
    if row=='oaklandraiders':
        return (37.751411,-122.20889)
    if row=="49ers":
        return (37.4026,-121.9694)
    if row=="LosAngelesRams":
        return (34.0141,118.2879)
    if row=="Chargers":
        return (33.864654, -118.261078)
    if row=='steelers':
        return (40.446786,-80.15761)
    if row=="eagles":
        return (39.900775,-75.167453)
    if row=='Tennesseetitans':
        return (36.166461,-86.771289)
    if row=='CHIBears':
        return (41.862306,-87.616672)
    if row=='falcons':
        return (33.757614,-84.400972)
    if row=='AZCardinals':
        return (33.5277,-112.262608)
    if row=='detroitlions':
        return (42.340156,-83.045808)
    if row=='panthers':
        return (35.225808,-80.852861)
    if row=='GreenBayPackers':
        return (44.501306,-88.062167)
    if row=="Saints":
        return (29.950931,-90.081364)
    if row=="Seahawks":
        return (47.595153,-122.331625)
    if row=='minnesotavikings':
        return (44.973881,-93.258094)

In [25]:
train['lat_long'] = train['subreddit'].apply(get_state)

In [26]:
# feather.write_dataframe(train, "nfl_processed.feather")

In [4]:
import feather
import pandas as pd

In [3]:
df_get = feather.read_dataframe("nfl_processed.feather")

In [5]:
df_get.head()

,_id,author,author_flair_text,body,can_gild,collapsed,collapsed_reason,controversiality,created_utc,distinguished,...,subreddit_id,neg,neu,pos,compound,datetime,state,month,year,dayofweek
0,ObjectId(5d585f95a772b7432fc82a65),NaN,NaN,A lot of anecdotes about a fired up defense ch...,True,False,None,0,1501545604,None,...,t5_2s2av,0.108,0.561,0.331,0.8519,2017-08-01 00:00:04,NY,8,2017,1
1,ObjectId(5d585f95a772b7432fc82b20),NaN,NaN,Nice. So based on this positive report he shou...,True,False,None,0,1501545609,None,...,t5_2s0u3,0.130,0.530,0.340,0.6542,2017-08-01 00:00:09,IL,8,2017,1
2,ObjectId(5d585f95a772b7432fc82c9b),NaN,NaN,"Future salaries would be a concern, but that's...",True,False,None,0,1501545620,None,...,t5_2re7g,0.183,0.619,0.198,0.0772,2017-08-01 00:00:20,TX,8,2017,1
3,ObjectId(5d585f95a772b7432fc82de8),NaN,NaN,PFF generally graded him as a below-average st...,True,False,None,0,1501545629,None,...,t5_2s450,0.148,0.702,0.150,0.3071,2017-08-01 00:00:29,TX,8,2017,1
4,ObjectId(5d585f95a772b7432fc8304f),NaN,NaN,"dude, you gotta risk it for the bojangles bisc...",True,False,None,0,1501545646,None,...,t5_2s1w0,0.149,0.851,0.000,-0.2732,2017-08-01 00:00:46,SC,8,2017,1


In [12]:
df_high_score = df_get.loc[df_get['score']>10]

In [13]:
df_high_score.shape

(1204909, 30)

In [20]:
feather.write_dataframe(df_high_score,"df_high_score.feather")

In [18]:
df_highest_score = df_get.loc[df_get['score']>30]

In [19]:
df_highest_score.shape

(293994, 30)

In [21]:
feather.write_dataframe(df_highest_score,"df_highest_score.feather")

In [22]:
df_high_score['body'].to_csv("high_scores_generate.csv")

/home/ubuntu/anaconda3/envs/ds_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [23]:
df_highest_score['body'].to_csv("highest_scores_generate.csv")

/home/ubuntu/anaconda3/envs/ds_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
